In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs4
%pylab inline
import requests
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
city_list = ["tippecanoe", "bloomington", "chicago", "cincinnati", "newyork", "detroit", "seattle", 
             "losangeles", "sfbay", "austin"]

links = []
for city in city_list:
  page = requests.get(f"https://{city}.craigslist.org/d/appliances/search/ppa")
  bsobj = bs4(page.content,'lxml')
  for link in bsobj.findAll('li',{'class':'result-row'}):
      links.append(link.a['href'])

In [3]:
scraped_data = pd.DataFrame({'link':links})
scraped_data.head()

,link
0,https://tippecanoe.craigslist.org/app/d/fisher...
1,https://tippecanoe.craigslist.org/app/d/lafaye...
2,https://tippecanoe.craigslist.org/app/d/lafaye...
3,https://tippecanoe.craigslist.org/app/d/fisher...
4,https://tippecanoe.craigslist.org/app/d/brooks...


In [325]:
def get_listing_info(args):
    link = args[0]
    listing_heading = 'NA'
    listing_price = 'NA'
    condition_val = 'NA'
    description = 'NA'
    page = requests.get(link)
    bsobj = bs4(page.content,'lxml')
    for link in bsobj.findAll('h1',{'class':'postingtitle'}):
        listing_heading = link.text
        heading_split = listing_heading.split()
        for tag in heading_split:
          if tag.startswith('$') and tag[-1].isdigit():
            listing_price = tag
    for link in bsobj.findAll('p',{'class':'attrgroup'}):
      temp_tag = link.text.split('\n')
      temp_tag = [val.strip() for val in temp_tag if val.startswith('condition')]
      if len(temp_tag) > 0:
        condition_val = temp_tag[0].split(':')[1].strip()
    for i in bsobj.findAll('section',{'id':'postingbody'}):
        description = i.text.strip().replace('QR Code Link to This Post','').replace('\n','')
    return pd.Series([listing_heading.strip(), description, listing_price, condition_val])

In [326]:
scraped_data[['Title', 'Description' ,'Price', 'Condition']] = scraped_data.apply(get_listing_info, axis=1,  result_type='expand')
scraped_data['Title'] = scraped_data['Title'].str.strip()

In [348]:
scraped_data.to_csv('craiglist_1k_appliances_data.csv', index=False)